In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!pip install folium
import folium # map rendering library

print('All libraries required are imported.')

All libraries required are imported.


In [5]:
CLIENT_ID = 'AUBT2WFNKKMY105SCBF0PUTKG45MVZTB44ELOCAZ4KEL305N' # your Foursquare ID
CLIENT_SECRET = 'PNTUXFEJLVUXLBZAIQU54GI0MRQ3BUXRZUYJE4OZ1Z52DJBO' # your Foursquare Secret
VERSION = '20180605'
print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Credentails:
CLIENT_ID: AUBT2WFNKKMY105SCBF0PUTKG45MVZTB44ELOCAZ4KEL305N


In [21]:
LIMIT = 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4d4b7105d754a06372d81259") 
    results[city] = requests.get(url).json()
    
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']
df_venues

{'New York, NY':                                                  Name                 Address  \
 0                                The Juilliard School   60 Lincoln Center Plz   
 1              New York College of Health Professions            22 E 30th St   
 2                 Fordham University - Lincoln Center           113 W 60th St   
 3                                      The New School            66 W 12th St   
 4                                        Cornell Tech             2 W Loop Rd   
 5                           The Graduate Center, CUNY             365 5th Ave   
 6                                     Pace University              1 Pace Plz   
 7                          The Rockefeller University           1230 York Ave   
 8                                  NYU Midtown Campus            11 W 42nd St   
 9                               Hunter College - CUNY            695 Park Ave   
 10                                New York University             25 W 4th St   


In [23]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of colleges and universities in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")



Total number of colleges and universities in New York, NY =  240
Showing Top 100
Total number of colleges and universities in Chicago, IL =  233
Showing Top 100
Total number of colleges and universities in San Francisco, CA =  163
Showing Top 100
Total number of colleges and universities in Jersey City, NJ =  119
Showing Top 100
Total number of colleges and universities in Boston, MA =  194
Showing Top 100


In [24]:
maps[cities[0]]

In [25]:
maps[cities[1]]

In [10]:

maps[cities[2]]

In [26]:
maps[cities[3]]

In [27]:
maps[cities[4]]

In [28]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.031587885208364724
Chicago, IL
Mean Distance from Mean coordinates
0.07144567056055597
San Francisco, CA
Mean Distance from Mean coordinates
0.03368401268711221
Jersey City, NJ
Mean Distance from Mean coordinates
0.014589505177495582
Boston, MA
Mean Distance from Mean coordinates
0.026636008411819935


In [30]:
maps[cities[0]]

In [31]:
maps[cities[1]]

In [16]:
maps[cities[2]]

In [32]:
maps[cities[3]]

In [33]:
maps[cities[4]]